In [55]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [56]:
df_nopass = pd.concat([pd.read_csv("results/out_nopass/faults_memory.csv"),
                       ])

df_eddi_cfcss = pd.concat([pd.read_csv("results/out_eddi_cfcss/faults_memory.csv"),
                           ]+
                           [pd.read_csv("results/out_eddi_cfcss/faults_memory_new.csv"),
                           ])

df_eddi_rasm_inter = pd.concat([pd.read_csv("results/out_eddi_rasm_inter/faults_memory.csv"),
                                ]+
                                [pd.read_csv("results/out_eddi_rasm_inter/faults_memory_new.csv"),
                                ])

df_eddi_rasm = pd.concat([pd.read_csv("results/out_eddi_rasm/faults_memory.csv"),
                          ]+
                          [pd.read_csv("results/out_eddi_rasm/faults_memory_new.csv"),
                          ])

df_fdsc_cfcss = pd.concat([pd.read_csv("results/out_fdsc_cfcss/faults_memory.csv"),
                           ]+
                           [pd.read_csv("results/out_fdsc_cfcss/faults_memory_new.csv"),
                           ])

df_fdsc_rasm_inter = pd.concat([pd.read_csv("results/out_fdsc_rasm_inter/faults_memory.csv"),
                                ]+
                                [pd.read_csv("results/out_fdsc_rasm_inter/faults_memory_new.csv"),
                                ])

df_fdsc_rasm = pd.concat([pd.read_csv("results/out_fdsc_rasm/faults_memory.csv"),
                          ]+
                          [pd.read_csv("results/out_fdsc_rasm/faults_memory_new.csv"),
                          ])

df_seddi_cfcss = pd.concat([pd.read_csv("results/out_seddi_cfcss/faults_memory.csv"),
                           ]+
                           [pd.read_csv("results/out_seddi_cfcss/faults_memory_new.csv"),
                           ])

df_seddi_rasm_inter = pd.concat([pd.read_csv("results/out_seddi_rasm_inter/faults_memory.csv"),
                                ]+
                                [pd.read_csv("results/out_seddi_rasm_inter/faults_memory_new.csv"),
                                ])

df_seddi_rasm = pd.concat([pd.read_csv("results/out_seddi_rasm/faults_memory.csv"),
                          ]+
                          [pd.read_csv("results/out_seddi_rasm/faults_memory_new.csv"),
                          ])


dataframes = {"eddi_cfcss": df_eddi_cfcss,
              "eddi_rasm": df_eddi_rasm,
              "eddi_rasm_inter": df_eddi_rasm_inter,
              "fdsc_cfcss": df_fdsc_cfcss,
              "fdsc_rasm": df_fdsc_rasm,
              "fdsc_rasm_inter": df_fdsc_rasm_inter,
              "seddi_cfcss": df_seddi_cfcss,
              "seddi_rasm": df_seddi_rasm,
              "seddi_rasm_inter": df_seddi_rasm_inter
              }

dataframes["nopass"] = df_nopass

df_new_m = pd.DataFrame()

for key in dataframes:
    df = dataframes[key]
    sz_tot = df.shape[0]
    df_errcodes = df.groupby('code')
    sz_df_errcodes = df_errcodes.size()
    df_new_m[key] = sz_df_errcodes

In [57]:
df_nopass = pd.concat([
                       pd.read_csv("results/out_nopass/faults_registers.csv")])

df_eddi_cfcss = pd.concat([
                           pd.read_csv("results/out_eddi_cfcss/faults_registers.csv")]+
                           [
                           pd.read_csv("results/out_eddi_cfcss/faults_registers_new.csv")])

df_eddi_rasm_inter = pd.concat([
                                pd.read_csv("results/out_eddi_rasm_inter/faults_registers.csv")]+
                                [
                                pd.read_csv("results/out_eddi_rasm_inter/faults_registers_new.csv")])

df_eddi_rasm = pd.concat([
                          pd.read_csv("results/out_eddi_rasm/faults_registers.csv")]+
                          [
                          pd.read_csv("results/out_eddi_rasm/faults_registers_new.csv")])

df_fdsc_cfcss = pd.concat([
                           pd.read_csv("results/out_fdsc_cfcss/faults_registers.csv")]+
                           [
                           pd.read_csv("results/out_fdsc_cfcss/faults_registers_new.csv")])

df_fdsc_rasm_inter = pd.concat([
                                pd.read_csv("results/out_fdsc_rasm_inter/faults_registers.csv")]+
                                [
                                pd.read_csv("results/out_fdsc_rasm_inter/faults_registers_new.csv")])

df_fdsc_rasm = pd.concat([
                          pd.read_csv("results/out_fdsc_rasm/faults_registers.csv")]+
                          [
                          pd.read_csv("results/out_fdsc_rasm/faults_registers_new.csv")])

df_seddi_cfcss = pd.concat([
                           pd.read_csv("results/out_seddi_cfcss/faults_registers.csv")]+
                           [
                           pd.read_csv("results/out_seddi_cfcss/faults_registers_new.csv")])

df_seddi_rasm_inter = pd.concat([
                                pd.read_csv("results/out_seddi_rasm_inter/faults_registers.csv")]+
                                [
                                pd.read_csv("results/out_seddi_rasm_inter/faults_registers_new.csv")])

df_seddi_rasm = pd.concat([
                          pd.read_csv("results/out_seddi_rasm/faults_registers.csv")]+
                          [
                          pd.read_csv("results/out_seddi_rasm/faults_registers_new.csv")])


dataframes = {"eddi_cfcss": df_eddi_cfcss,
              "eddi_rasm": df_eddi_rasm,
              "eddi_rasm_inter": df_eddi_rasm_inter,
              "fdsc_cfcss": df_fdsc_cfcss,
              "fdsc_rasm": df_fdsc_rasm,
              "fdsc_rasm_inter": df_fdsc_rasm_inter,
              "seddi_cfcss": df_seddi_cfcss,
              "seddi_rasm": df_seddi_rasm,
              "seddi_rasm_inter": df_seddi_rasm_inter
              }

dataframes["nopass"] = df_nopass

In [58]:
df_new_r = pd.DataFrame()

for key in dataframes:
    df = dataframes[key]
    sz_tot = df.shape[0]
    df_errcodes = df.groupby('code')
    sz_df_errcodes = df_errcodes.size()
    df_new_r[key] = sz_df_errcodes

In [59]:
df_perc_m = (df_new_m.drop(0) / df_new_m.drop(0).sum() * 100)
df_perc_r = (df_new_r.drop(0) / df_new_r.drop(0).sum() * 100)

In [60]:
((df_perc_m * df_new_m.drop(0).sum() + df_perc_r * df_new_r.drop(0).sum()/1250) / (df_new_m.drop(0).sum() + df_new_r.drop(0).sum()/1250)).transpose().to_csv("weighted_res.csv")

In [65]:
P_m = 1280 / 1281
F_m = df_new_m.sum()
F_i_m = df_new_m
F_E_m = df_new_m.sum()

P_r = 1 / 1281
F_r = df_new_r.sum()
F_i_r = df_new_r
F_E_r = df_new_r.sum()

res = (P_m * F_i_m / F_m  + P_r * F_i_r/F_r) / (P_m * F_E_m / F_m  + P_r * F_E_r/F_r) * 100
res.round(4).transpose()

code,-3,-2,-1,0,1,2
eddi_cfcss,0.5910,0.0118,1.3273,94.8902,2.6706,0.5090
eddi_rasm,0.5527,0.0161,1.3831,95.0859,2.6420,0.3202
eddi_rasm_inter,0.5559,0.0050,1.8043,94.5645,2.6089,0.4614
fdsc_cfcss,0.6836,0.0374,1.6485,95.5894,1.5735,0.4677
fdsc_rasm,0.6580,0.0582,1.6109,95.8334,1.5662,0.2733
fdsc_rasm_inter,0.6168,0.0299,1.9209,95.4652,1.5198,0.4473
seddi_cfcss,0.6144,0.0223,1.4254,95.2349,2.2253,0.4776
seddi_rasm,0.6124,0.0119,1.4033,95.3263,2.2903,0.3559
seddi_rasm_inter,0.6072,0.0098,1.9076,94.5653,2.4257,0.4844
nopass,0.9737,0.0652,1.2992,97.6619,NaN,NaN


In [68]:
2.79 / 0.09

31.0

In [61]:
P_m = 1280 / 1281
F_m = df_new_m.sum()
F_i_m = df_new_m.drop(0)
F_E_m = df_new_m.drop(0).sum()

P_r = 1 / 1281
F_r = df_new_r.sum()
F_i_r = df_new_r.drop(0)
F_E_r = df_new_r.drop(0).sum()

res = (P_m * F_i_m / F_m  + P_r * F_i_r/F_r) / (P_m * F_E_m / F_m  + P_r * F_E_r/F_r) * 100
res.round(2).transpose().to_csv("weighted_res.csv")
for elem in res:
    print(res[elem].sum())

100.0
100.00000000000001
100.0
100.00000000000003
99.99999999999999
99.99999999999999
100.0
100.0
100.0
100.0


In [62]:
((1280 / 1281 * df_new_m.drop(0) + 1 / 1281 * df_new_r.drop(0))/(1280 / 1281 * df_new_m.drop(0).sum() + 1 / 1281 * df_new_r.drop(0).sum())).transpose() * 100

code,-3,-2,-1,1,2
eddi_cfcss,11.570448,0.231485,25.990796,52.291844,9.915427
eddi_rasm,11.252335,0.328494,28.166585,53.794669,6.457917
eddi_rasm_inter,10.237575,0.091795,33.203041,48.015021,8.452568
fdsc_cfcss,15.504365,0.848135,37.402022,35.703102,10.542375
fdsc_rasm,15.806007,1.398915,38.692144,37.620926,6.482009
fdsc_rasm_inter,13.616247,0.658061,42.374635,33.535505,9.815552
seddi_cfcss,12.897773,0.469051,29.930847,46.731013,9.971315
seddi_rasm,13.110467,0.255293,30.048134,49.035789,7.550318
seddi_rasm_inter,11.182200,0.180547,35.104624,44.654370,8.878259
nopass,41.638673,2.788113,55.573214,NaN,NaN


In [63]:
df_new_m.drop(0).sum()

eddi_cfcss          1296.0
eddi_rasm           1217.0
eddi_rasm_inter     1093.0
fdsc_cfcss          1296.0
fdsc_rasm           1214.0
fdsc_rasm_inter     1064.0
seddi_cfcss         1279.0
seddi_rasm          1174.0
seddi_rasm_inter    1108.0
nopass               466.0
dtype: float64